<a href="https://colab.research.google.com/github/billsioros/thesis/blob/master/Nanorough_surface_Super_resolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ✔️ Prerequisites

First of all we need to take care of a few **prerequisites**, most notably:

- Install the various pip modules that we will be using.
- Install some linux specific dependencies of our [content loss](#content-loss).
- Initialize the Random Number Generator(s), so that our experiments can be replicated.
- Determine:
  - The current working directory, as it's going to be used to reference various files such as the dataset, our model checkpoints e.t.c
  - The available hardware backend. GPU utilization is preferable, as it results in higher complition time.
- `(Optionally)` Mount Google Drive, where we can load our dataset from.

## Installing [graphviz](https://graphviz.org/) & [libgraphviz-dev](https://packages.debian.org/jessie/libgraphviz-dev)

The aforementioned packages are required by [PyINSECT](https://github.com/billsioros/PyINSECT/tree/implementing-HPGs) and more specifically its graph plotting methods.

In [1]:
!sudo apt-get install graphviz libgraphviz-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
graphviz is already the newest version (2.40.1-2).
The following additional packages will be installed:
  libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin libgtk2.0-common
  libgvc6-plugins-gtk libxdot4
Suggested packages:
  gvfs
The following NEW packages will be installed:
  libgail-common libgail18 libgraphviz-dev libgtk2.0-0 libgtk2.0-bin
  libgtk2.0-common libgvc6-plugins-gtk libxdot4
0 upgraded, 8 newly installed, 0 to remove and 39 not upgraded.
Need to get 2,120 kB of archives.
After this operation, 7,128 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libgtk2.0-common all 2.24.32-1ubuntu1 [125 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libgtk2.0-0 amd64 2.24.32-1ubuntu1 [1,769 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/main amd64 libgail18 amd64 2.24.32-1ubuntu1 [14.2 kB]
Get:4 http://archive.ubuntu.com/ub

## Installing the required `pip` modules

- [torch](https://pytorch.org/) is our machine learning framework of choice.
- [numpy](https://numpy.org/), [sympy](https://www.sympy.org/en/index.html) and [scipy](https://www.scipy.org/) are used to in the context of nanorough surface generation.
- [plotly](https://plotly.com/) (which requires [pandas](https://pandas.pydata.org/)) as well as [matplotlib](https://matplotlib.org/) are used in order to plot various graphs.

In [13]:
!pip install torch numpy sympy scipy plotly pandas sklearn matplotlib==3.1.1 git+https://github.com/billsioros/PyINSECT.git@FEATURE_Implementing_HPGraphCollector git+https://github.com/billsioros/thesis.git

  Cloning https://****@github.com/billsioros/PyINSECT.git (to revision FEATURE_Implementing_HPGraphCollector) to /tmp/pip-req-build-znlw5e9i
  Running command git clone -q 'https://****@github.com/billsioros/PyINSECT.git' /tmp/pip-req-build-znlw5e9i
  Running command git checkout -b FEATURE_Implementing_HPGraphCollector --track origin/FEATURE_Implementing_HPGraphCollector
  Switched to a new branch 'FEATURE_Implementing_HPGraphCollector'
  Branch 'FEATURE_Implementing_HPGraphCollector' set up to track remote branch 'FEATURE_Implementing_HPGraphCollector' from 'origin'.
  Cloning https://github.com/billsioros/thesis.git to /tmp/pip-req-build-mmq0uvgx
  Running command git clone -q https://github.com/billsioros/thesis.git /tmp/pip-req-build-mmq0uvgx
  fatal: could not read Username for 'https://github.com': No such device or address
ERROR: Command errored out with exit status 128: git clone -q https://github.com/billsioros/thesis.git /tmp/pip-req-build-mmq0uvgx Check the logs for full co

## Initializing (a.k.a `Seeding`) the Random Number Generator(s)

We are required to seed various random number generation engines, so that our experiments can be replicated on a later date.

In [3]:
SEED = 1234

import os
import random

import numpy as np

In [4]:
import torch

if SEED is not None:
    np.random.seed(SEED)
    random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True
    os.environ["PYTHONHASHSEED"] = str(SEED)

## Determining the Current Working Directory

In [5]:
from pathlib import Path

BASE_DIR = Path.cwd()

## Mounting Google Drive

In [6]:
GDRIVE_DIR = BASE_DIR / "drive"

In [7]:
try:
    from google.colab import drive

    drive.mount(f"{GDRIVE_DIR}")
except ImportError:
    pass

Mounted at /content/drive


## Determining available backend

By default, we are going to be utilizing the available CPU backend, if no GPU is available.

In [8]:
device = "cpu"
if torch.cuda.is_available():
    device = "cuda:0"

In [9]:
device = torch.device(device)

## Configuring our Loggers

In [10]:
import logging

logging.basicConfig(
    format="[%(asctime)s] %(levelname)s:%(name)s: %(message)s", level=logging.CRITICAL
)

In [11]:
logger = logging.getLogger()

# 🙃 A naive-approach

## Instantiating the **Generator** and the **Discriminator** Networks

In [12]:
from roughml.models import PerceptronGenerator

generator = PerceptronGenerator.from_device(device)

ModuleNotFoundError: ignored

In [ ]:
generator

In [ ]:
from roughml.models import PerceptronDiscriminator

discriminator = PerceptronDiscriminator.from_generator(generator, device=device)

In [ ]:
discriminator

## Training

In [ ]:
from torch.nn import BCELoss

criterion = BCELoss().to(device)

In [ ]:
from pathlib import Path

CHECKPOINT_DIR = BASE_DIR / "checkpoint"
CHECKPOINT_DIR.mkdir(parents=True, exist_ok=True)

from roughml.content.loss import NGramGraphContentLoss
from roughml.data.transforms import Flatten, To

In [ ]:
from roughml.training.flow import TrainingFlow
from roughml.training.manager import per_epoch

training_flow = TrainingFlow(
    training_manager={
        "benchmark": True,
        "checkpoint_dir": CHECKPOINT_DIR,
        "checkpoint_multiple": False,
        "train_epoch": per_epoch,
        "log_every_n": 10,
        "criterion": criterion,
        "n_epochs": 10,
        "train_ratio": 0.8,
        "optimizer": {"lr": 0.0005, "weight_decay": 0},
        "dataloader": {
            "batch_size": 256,
            "shuffle": True,
            "num_workers": 0,
        },
    },
    content_loss_type=NGramGraphContentLoss,
    dataset={
        "limit": 10,
        "path": GDRIVE_DIR / "MyDrive" / "Thesis" / "Datasets" / "surfaces.zip",
        "transforms": [Flatten(), To(device)],
    },
)

In [ ]:
training_flow(generator, discriminator)

# 😎 A CNN based approach

## Instantiating the **Generator** and the **Discriminator** Networks

In [ ]:
from src.roughml.models import CNNGenerator

generator = CNNGenerator.from_device(device)

In [ ]:
generator

In [ ]:
from src.roughml.models import CNNDiscriminator

discriminator = CNNDiscriminator.from_device(device)

In [ ]:
discriminator

## Training

In [ ]:
from torch.nn import BCELoss

criterion = BCELoss().to(device)

from roughml.content.loss import ArrayGraph2DContentLoss
from roughml.data.transforms import To, View

In [ ]:
from roughml.training.flow import TrainingFlow
from roughml.training.manager import per_epoch

training_flow = TrainingFlow(
    training_manager={
        "benchmark": True,
        "checkpoint_dir": CHECKPOINT_DIR,
        "checkpoint_multiple": False,
        "train_epoch": per_epoch,
        "log_every_n": 10,
        "criterion": criterion,
        "n_epochs": 10,
        "train_ratio": 0.8,
        "optimizer": {"lr": 0.0002, "betas": (0.5, 0.999)},
        "dataloader": {
            "batch_size": 256,
            "shuffle": True,
            "num_workers": 0,
        },
    },
    content_loss_type=ArrayGraph2DContentLoss,
    dataset={
        "limit": 10,
        "path": GDRIVE_DIR / "MyDrive" / "Thesis" / "Datasets" / "surfaces.zip",
        "transforms": [To(device), View(1, 128, 128)],
    },
)

In [ ]:
training_flow(generator, discriminator)